In [2]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd,numpy as np

## Scrapping the data

In [7]:
yield_data = []
from_year=2001
to_year=2023
for i in range(from_year,to_year+1):
    url = f"https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value={i}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")


    table = soup.find("table")
    rows = table.find_all("tr")

    yield_data+=list(map(lambda x:re.sub(r' +',"",x.text.replace('N/A','')),rows))
    yield_data

## Data Cleaning

In [1]:
df=pd.DataFrame([sub.split('\n') for sub in yield_data]).drop_duplicates()                                   # Creating a DataFrame from the yield data and dropping any duplicate rows
new_header = df.iloc[0]                                                                                      # Selecting the first row as the new column headers
df = df[1:]                                                                                                  # Excluding the first row from the DataFrame as it is used for column headers
df.columns = new_header                                                                                      # Assigning the first row as the column headers
df['Date']=pd.to_datetime(df['Date'])                                                                        # Converting the 'Date' column to datetime format
df.set_index('Date', inplace=True)                                                                           # Setting the 'Date' column as the index of the DataFrame
df=df.replace('',pd.NA).dropna(how='all', axis=1)                                                            # Replacing empty strings with NA and dropping columns where all values are NA
df.columns = ["1 Mo",'2 Mo','3 Mo','4 Mo','6 Mo','1 Yr','2 Yr','3 Yr','5 Yr','7 Yr','10 Yr','20 Yr','30 Yr'] # Renaming the columns for clarity
df=df.apply(pd.to_numeric)                                                                                   # Converting all columns to numeric data type
df 


NameError: name 'pd' is not defined

In [ ]:
df

In [9]:
df.to_csv(f'yield_curve{from_year}-{to_year}.csv')